In [6]:
import os
import numpy as np
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet import preprocess_input
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# Initialize the pre-trained MobileNet model without the top classification layer
model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3), pooling='avg')

# Function to extract features from an image
def extract_features(img_path, model):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)  # Preprocess for MobileNet
    features = model.predict(img_array)
    return features.flatten()

# Directory paths
pizza_train_dir = '../data/food11/train/pizza'
pizza_test_dir = '../data/food11/test/pizza'

# Collect all image paths in the pizza folders
image_paths = [os.path.join(pizza_train_dir, fname) for fname in os.listdir(pizza_train_dir)]
image_paths += [os.path.join(pizza_test_dir, fname) for fname in os.listdir(pizza_test_dir)]

# Extract features for all images
features = []
for img_path in image_paths:
    try:
        features.append(extract_features(img_path, model))
    except Exception as e:
        print(f"Error processing image {img_path}: {e}")

features = np.array(features)
print(f"Extracted features for {len(features)} images")

Extracted features for 1000 images


In [7]:
from sklearn.cluster import KMeans

# Number of clusters (choose based on expected subcategories, e.g., 3 types of pizza)
num_clusters = 3

# Perform K-Means clustering
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(features)

# Get cluster labels for each image
cluster_labels = kmeans.labels_

# Print the cluster label for each image
for img_path, label in zip(image_paths, cluster_labels):
    print(f"{img_path} assigned to cluster {label}")

c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


../data/food11/train/pizza\1001116.jpg assigned to cluster 1
../data/food11/train/pizza\1008104.jpg assigned to cluster 2
../data/food11/train/pizza\1008144.jpg assigned to cluster 2
../data/food11/train/pizza\1008844.jpg assigned to cluster 1
../data/food11/train/pizza\1008941.jpg assigned to cluster 0
../data/food11/train/pizza\1011404.jpg assigned to cluster 1
../data/food11/train/pizza\1026922.jpg assigned to cluster 1
../data/food11/train/pizza\1029698.jpg assigned to cluster 2
../data/food11/train/pizza\1032754.jpg assigned to cluster 2
../data/food11/train/pizza\1033251.jpg assigned to cluster 2
../data/food11/train/pizza\1035854.jpg assigned to cluster 2
../data/food11/train/pizza\1038357.jpg assigned to cluster 0
../data/food11/train/pizza\1040878.jpg assigned to cluster 0
../data/food11/train/pizza\1044524.jpg assigned to cluster 0
../data/food11/train/pizza\1044789.jpg assigned to cluster 1
../data/food11/train/pizza\1047561.jpg assigned to cluster 0
../data/food11/train/piz

In [8]:
import shutil

# Create directories for each cluster (subcategories)
for i in range(num_clusters):
    train_subdir = os.path.join(pizza_train_dir, f"cluster_{i}")
    test_subdir = os.path.join(pizza_test_dir, f"cluster_{i}")
    os.makedirs(train_subdir, exist_ok=True)
    os.makedirs(test_subdir, exist_ok=True)

# Move images to subdirectories based on cluster labels
for img_path, label in zip(image_paths, cluster_labels):
    if 'train' in img_path:
        shutil.move(img_path, os.path.join(pizza_train_dir, f"cluster_{label}", os.path.basename(img_path)))
    elif 'test' in img_path:
        shutil.move(img_path, os.path.join(pizza_test_dir, f"cluster_{label}", os.path.basename(img_path)))

print("Images moved to their respective cluster directories.")

Images moved to their respective cluster directories.


In [9]:
# Reuse your previous model setup for fine-grained classification
pizza_train_generator = pizza_train_gen.flow_from_directory(
    pizza_train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

pizza_test_generator = pizza_test_gen.flow_from_directory(
    pizza_test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Train the fine-grained pizza classification model
pizza_model.fit(
    pizza_train_generator,
    steps_per_epoch=pizza_train_generator.samples // 32,
    validation_data=pizza_test_generator,
    validation_steps=pizza_test_generator.samples // 32,
    epochs=30
)

# Save the model
pizza_model.save('fine_grained_pizza_classifier.h5')

Found 900 images belonging to 3 classes.
Found 100 images belonging to 3 classes.
Epoch 1/30
28/28 [==============================] - 14s 466ms/step - loss: 0.7699 - accuracy: 0.6694 - val_loss: 0.4131 - val_accuracy: 0.8646
Epoch 2/30
28/28 [==============================] - 11s 408ms/step - loss: 0.2935 - accuracy: 0.9182 - val_loss: 0.2838 - val_accuracy: 0.9167
Epoch 3/30
28/28 [==============================] - 11s 413ms/step - loss: 0.1939 - accuracy: 0.9551 - val_loss: 0.2264 - val_accuracy: 0.9271
Epoch 4/30
28/28 [==============================] - 11s 397ms/step - loss: 0.1446 - accuracy: 0.9735 - val_loss: 0.2153 - val_accuracy: 0.9167
Epoch 5/30
28/28 [==============================] - 11s 403ms/step - loss: 0.1206 - accuracy: 0.9839 - val_loss: 0.1909 - val_accuracy: 0.9479
Epoch 6/30
28/28 [==============================] - 12s 419ms/step - loss: 0.1037 - accuracy: 0.9827 - val_loss: 0.1709 - val_accuracy: 0.9271
Epoch 7/30
28/28 [==============================] - 12s 412m